## 머신러닝 모델 이용

## 라이브러리 로드

In [1]:
import pandas as pd
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

RANDOM_STATE = 110

# 경고 메시지 억제
warnings.filterwarnings('ignore')

데이터셋 불러오기 : 영문기사 제거(+html 제거) 데이터셋 사용

In [2]:
df_train = pd.read_csv("../../data/train_df_translate_del.csv")
df_test = pd.read_csv("../../data/test_df_translate_del.csv")

전처러 1. 중복 행 제거

In [3]:
df_train.drop_duplicates(subset=['제목', '키워드'], keep='first', inplace=True)

전처러 2. url 주소 제거

In [4]:
import pandas as pd
import re

# 제거할 단어 리스트
remove_words = ['http', 'www.', '.kr', '.net', '.com']

def remove_specific_words(text):
    words = text.split(',')
    filtered_words = [word for word in words if not any(remove_word in word for remove_word in remove_words)]
    return ','.join(filtered_words)

# '키워드' 열에 함수 적용
df_train['키워드'] = df_train['키워드'].apply(remove_specific_words)
df_test['키워드'] = df_test['키워드'].apply(remove_specific_words)

In [5]:
df_train.shape, df_test.shape

((54315, 4), (23405, 3))

## 앙상블 

In [ ]:
# 텍스트와 레이블 분리
X = df_train['키워드']  # 키워드 컬럼
y = df_train['분류']  # 카테고리 컬럼

# 데이터 분할 (클래스 비율 동일하게 유지)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)

# Count Vectorization
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_valid_count = count_vectorizer.transform(X_valid)

# 반복 횟수 및 규제 강도 설정
max_iters = [50, 100, 150, 200, 250, 300]
Cs = [0.01, 0.1, 1, 10, 100]

for max_iter in max_iters:
    for C in Cs:
        # 로지스틱 회귀 모델 정의
        model_count = LogisticRegression(random_state=RANDOM_STATE, max_iter=max_iter, C=C)

        # 모델 학습
        model_count.fit(X_train_count, y_train_encoded)

        # 예측
        y_valid_pred = model_count.predict(X_valid_count)

        # 성능 평가
        accuracy = accuracy_score(y_valid_encoded, y_valid_pred)
        macro_f1 = f1_score(y_valid_encoded, y_valid_pred, average='macro')
        print(f"Count Vectorization + Logistic Regression (max_iter={max_iter}, C={C}) - Accuracy: {accuracy}, Macro F1-score: {macro_f1}")

Count Vectorization + Logistic Regression (max_iter=50, C=0.01) - Accuracy: 0.8196630764982049, Macro F1-score: 0.6191453439964156
Count Vectorization + Logistic Regression (max_iter=50, C=0.1) - Accuracy: 0.8390868084322931, Macro F1-score: 0.6733327968773705
Count Vectorization + Logistic Regression (max_iter=50, C=1) - Accuracy: 0.8389026972291264, Macro F1-score: 0.6739632528416808
Count Vectorization + Logistic Regression (max_iter=50, C=10) - Accuracy: 0.8373377520022093, Macro F1-score: 0.6735982794742669
Count Vectorization + Logistic Regression (max_iter=50, C=100) - Accuracy: 0.8344840283531253, Macro F1-score: 0.6709326306291497
Count Vectorization + Logistic Regression (max_iter=100, C=0.01) - Accuracy: 0.8190186872871215, Macro F1-score: 0.6134241321173789
Count Vectorization + Logistic Regression (max_iter=100, C=0.1) - Accuracy: 0.84009942004971, Macro F1-score: 0.6737647361372543
Count Vectorization + Logistic Regression (max_iter=100, C=1) - Accuracy: 0.840743809260793

Count Vectorization + Logistic Regression (max_iter=100, C=1)     
- Accuracy: 0.8407438092607935, Macro F1-score: 0.6811214132075255    
- max_iter : (default) 100   
- C : (default) 1   

## 모든 데이터로 학습하여 예측

아래 코드만 total 코드에 실행시키면 됨

In [1]:
import pandas as pd
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# ----- 1. 데이터 불러오기 ----- #

RANDOM_STATE = 110

# 경고 메시지 억제
warnings.filterwarnings('ignore')

# 영문기사 및 html 문 제거한 데이터 사용
df_train = pd.read_csv("../../data/train_df_translate_del.csv")
df_test = pd.read_csv("../../data/test_df_translate_del.csv")


# ----- 2. 전처리 ----- #

## 전처리 1. 중복 제거
df_train.drop_duplicates(subset=['제목', '키워드'], keep='first', inplace=True)

## 전처리 2. url 제거
remove_words = ['http', 'www.', '.kr', '.net', '.com'] # 제거할 단어 리스트

def remove_specific_words(text):
    words = text.split(',')
    filtered_words = [word for word in words if not any(remove_word in word for remove_word in remove_words)]
    return ','.join(filtered_words)

# ----- 3. 모델링(학습) ----- #

# '키워드' 열에 함수 적용
df_train['키워드'] = df_train['키워드'].apply(remove_specific_words)
df_test['키워드'] = df_test['키워드'].apply(remove_specific_words)

# 텍스트와 레이블 분리
X_train = df_train['키워드']  # 키워드 컬럼
y_train = df_train['분류']  # 카테고리 컬럼
X_test = df_test['키워드']

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Count Vectorization
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# 로지스틱 회귀 모델 정의
model_count = LogisticRegression(random_state=RANDOM_STATE)

# 모델 학습
model_count.fit(X_train_count, y_train_encoded)

# ----- 4. 모델링(예측) ----- #
# 예측
count_predictions = model_count.predict(X_test_count)   

# 예측 결과를 데이터프레임으로 변환
count_results = pd.DataFrame({'ID': df_test['ID'], '분류': label_encoder.inverse_transform(count_predictions)})

# 결과를 CSV 파일로 저장
count_results.to_csv("1016_count_vectorization_lr_predictions.csv", index=False, encoding='utf-8-sig')

# 저장된 결과 확인
print("CSV 파일로 저장된 결과:")
print(count_results.head())

CSV 파일로 저장된 결과:
           ID        분류
0  TEST_00000        지역
1  TEST_00001   사회:사회일반
2  TEST_00002  정치:행정_자치
3  TEST_00003  경제:취업_창업
4  TEST_00004        지역


.